In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, to_date, sum, col, year, month, avg

spark = SparkSession.builder.appName("Weather").getOrCreate()

df_actors = spark.read.csv("/home/jovyan/work/PySpark_test/config/actors.csv", header=True, inferSchema=True)
df_movies = spark.read.csv("/home/jovyan/work/PySpark_test/config/movies.csv", header=True, inferSchema=True)
df_movie_actors = spark.read.csv("/home/jovyan/work/PySpark_test/config/movie_actors.csv", header=True, inferSchema=True)

df_actors.createOrReplaceTempView("actors")
df_movies.createOrReplaceTempView("movies")
df_movie_actors.createOrReplaceTempView("movie_actors")

top5_genre = spark.sql("""
SELECT genre, COUNT(title) AS num_movies
FROM movies
GROUP BY genre
ORDER BY num_movies DESC
""")

top_actor = spark.sql("""
SELECT a.name, COUNT(ma.movie_id) AS num_movies
FROM actors a
JOIN movie_actors ma ON
        a.actor_id = ma.actor_id
GROUP BY a.name
ORDER BY num_movies DESC
LIMIT 1
""")

avg_genre_budget = spark.sql("""
SELECT genre, AVG(budget) AS avg_budget
FROM movies
GROUP BY genre
""")

more_one_actor = spark.sql("""
SELECT m.title, a.country, COUNT(name) AS num_actors
FROM movies m
JOIN movie_actors ma ON
    m.movie_id = ma.movie_id
JOIN actors a ON
    a.actor_id = ma.actor_id
GROUP BY m.title, a.country
HAVING num_actors > 1
""")

# top5_genre.show()
# top_actor.show()
# avg_genre_budget.show()
# more_one_actor.show()
# df_movies.show()
# df_movie_actors.show()
spark.stop()